In [1]:
import sys
import numpy as np
import operator
import cv2
import os

In [2]:
# Module level variables: 

MIN_CONTOUR_AREA = 100
RESIZED_IMAGE_WIDTH  = 20
RESIZED_IMAGE_HEIGHT = 30

In [15]:
import cv2
import os
import numpy as np

MIN_CONTOUR_AREA = 100
RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30

def main():
    imgTrainingNumbers = cv2.imread('training_numbers.jfif')
    
    if imgTrainingNumbers is None:
        print("Error: Image not read from file")
        os.system("pause")
        return
    
    imgGray = cv2.cvtColor(imgTrainingNumbers, cv2.COLOR_BGR2GRAY)
    imgBlurred = cv2.GaussianBlur(imgGray, (5, 5), 0)
    
    imgThresh = cv2.adaptiveThreshold(imgBlurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    
    imgThreshCopy = imgThresh.copy()
    
    npaContours, _ = cv2.findContours(imgThreshCopy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    npaFlattenedImages = np.empty((0, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))
    intClassifications = []
    
    intValidChars = [ord('0'), ord('1'), ord('2'), ord('3'), ord('4'), ord('5'), ord('6'), ord('7'), ord('8'), ord('9'),
                     ord('A'), ord('B'), ord('C'), ord('D'), ord('E'), ord('F'), ord('G'), ord('H'), ord('I'), ord('J'),
                     ord('K'), ord('L'), ord('M'), ord('N'), ord('O'), ord('P'), ord('Q'), ord('R'), ord('S'), ord('T'),
                     ord('U'), ord('V'), ord('W'), ord('X'), ord('Y'), ord('Z')]
    
    for npaContour in npaContours:
        if cv2.contourArea(npaContour) > MIN_CONTOUR_AREA:
            [intX, intY, intW, intH] = cv2.boundingRect(npaContour)
            
            cv2.rectangle(imgTrainingNumbers, (intX, intY), (intX + intW, intY + intH), (0, 0, 255), 2)
            
            imgROI = imgThresh[intY:intY + intH, intX:intX + intW]
            imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))
            
            cv2.imwrite(f"ROI_{intX}_{intY}.png", imgROI)
            cv2.imwrite(f"ROI_Resized_{intX}_{intY}.png", imgROIResized)
            
            intChar = input("Enter the character: ")
            
            if intChar == 'exit':
                break
            elif intChar in intValidChars:
                intClassifications.append(ord(intChar))
                
                npaFlattenedImage = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))
                npaFlattenedImages = np.append(npaFlattenedImages, npaFlattenedImage, 0)
    
    fltClassifications = np.array(intClassifications, np.float32)
    npaClassifications = fltClassifications.reshape((fltClassifications.size, 1))
    
    print("Training complete!")
    
    np.savetxt("Classifications.txt", npaClassifications)
    np.savetxt("Flattened_images.txt", npaFlattenedImages)

if __name__ == "__main__":
    main()


Enter the character: ABC142
Enter the character: NKSG12
Enter the character: DSDFJB352
Enter the character: JHSADHJV78247
Enter the character: DSBVHGV234
Enter the character: SDFNHFVHV2545
Enter the character: SMNBFDNB2552
Enter the character: exit
Training complete!


In [5]:
# Test and Train:

class ContourWithData():
    # member variables:
    npaContour = None
    boundingRect = None
    intRectX = 0
    intRectY = 0
    intRectWidth = 0
    intRectHeight = 0
    fltArea = 0.0
    
    def calculate_Rect_Top_Left_Point_And_Width_And_Height(self):
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight
        
    def checkIfContourisValid(self):
        if self.fltArea < MIN_CONTOUR_AREA : return False
        return True

In [17]:
def main():
    allContoursWithData = []
    validContoursWithData = []
    
    try:
        npaClassifications = np.loadtxt("Classifications.txt", np.float32)
    except:
        print("Error: Unable to open file")
        os.system("pause")
        return
    
    npaClassifications = npaClassifications.reshape(npaClassifications.size, 1)
    
    # KNN model:
    
    kNearest = cv2.ml.KNearest_create()
    
    kNearest.train(npaFlattenedImages, cv2.ml.ROW_SAMPLE, npaClassifications)
    
    imgTestingNumbers = cv2.imread("image.png")
    
    if imgTestingNumbers is None:                                 # if image not read successfully, print error
        print("Error: Image not read from file")
        os.system("pause")
        return
    
    imgGray = cv2.cvtColor(imgTestingNumbers, cv2.COLOR_BGR2GRAY)    # get grayscale image
    imgBlurred = cv2.GaussianBlur(imgGray, (5,5), 0)                  # Blur
    
    imgThresh = cv2.adaptiveThreshold(imgBlurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    
    imgThreshCopy = imgThresh.copy()
    
    imgContours, npaContours, npaHierachy = cv2.findContours(imgThreshCopy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for npaContour in npaContours:
        contourWithData = ContourWithData()
        contourWithData.npaContour = npaContour
        contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)
        contourWithData.calculate_Rect_Top_Left_Point_And_Width_And_Height()
        contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)
        allContoursWithData.append(contourWithData)
        
    for contourWithData in allContoursWithData:
        if contourWithData.checkIfContourisValid():
            validContoursWithData.append(contourWithData)
            
    validContoursWithData.sort(key = operator.attrgetter("intRectX"))
    
    strFinalString = ""
    
    for contourWithData in validContoursWithData:
        cv2.rectangle(imgTestingNumbers, (contourWithData.intRectX, contourWithData.intRectY), (contourWithData.intRectX + contourWithData.intRectWidth, contourWithData.intRectY + contourWithData.intRectHeight), (0,255,0),2)
        
        imgROI = imgThresh[contourWithData.intRectY:contourWithData.intRectY+contourWithData.intRectHeight, contourWithData.intRectX:contourWithData.intRectX+contourWithData.intRectWidth]
        imgROIResized = cv2.resized(imgROI, RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT)
        
        npaROIResized = imgROIResized.reshape(1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT)
        npaROIResized = np.float32(npaROIResized)
        
        retval, npaResults, neigh_resp, dists = kNearest.findNearest(npaROIResized, k=1)
        
        strCurrentChar = str(char(int(npaResults[0][0])))
        
        strFinalString = strFinalString + strCurrentChar
        
        
    print(strFinalString)
    
    cv2.imshow("imgTestingNumbers", imgTestingNumbers)
    cv2.waitKey(0)
    
    cv2.destroyAllWindows()
    
    return
    
if __name__ == "__main__":
    main()